<a href="https://colab.research.google.com/github/seunghyeon98/NLP/blob/main/clf_finetuning_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661774 sha256=3580f792b9272e9512def079f969ffad6113893b6b06b46822dee610c38fbf0c
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-zkfaowq6/kobert-tokenizer_ce72afb46cd14c3eb7bf9dd03f4cf2c4
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-zkfaowq6/kobert-tokenizer_ce72afb46cd14c3eb7bf9dd03f4cf2c4
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=0ef3771536fc024aed72b80ed85339fbd7c55b2dcc804d678582fe8824c70700
  Stored in directory: /tmp/pip-ephem-wheel-cache-o9wnhm8l/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# device = torch.device("cuda:0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print device availability
print("Using device:", device)

Using device: cuda


In [7]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2024-02-12 09:55:59--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6028:18::a27d:4712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2024-02-12 09:55:59--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca7197601bea746f40326e937cc.dl.dropboxusercontent.com/cd/0/get/CNKp_RVDOIHDIT3uM8hkYMBHJVB0vV4IcrqkuIrRPaSlmiu16Cj-wK2C_Vj7ugUsWmqX2bwkyeLlJnsZKxc95Buh-kIbOef2xwYmsKw_69d5iutfwvAX0zDb74oiGOcoZtihEq6Y3gihaujTaF4fQK7P/file?dl=1# [following]
--2024-02-12 09:56:00--  https://uca7197601bea746f40326e937cc.dl.dropboxusercontent.com/cd/0/get/CNKp_RVDOIHDIT3uM8hkYMBHJVB0vV4IcrqkuIrRPaSlmiu16Cj-wK2C_Vj7ugUsWmqX2bwkyeLlJnsZKx

In [30]:
import pandas as pd

# Load training data
train_data = pd.read_csv('ratings_train.txt?dl=1', sep='\t')

# Load testing data
test_data = pd.read_csv('ratings_test.txt?dl=1', sep='\t')

In [31]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [10]:
train_data.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [32]:
train_data.shape

(150000, 3)

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [51]:
max_length = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
dr_rate = 0.5

In [74]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self,data,tokenizer,max_length):
    self.data = data
    self.tokenizer = tokenizer
    self.max_length = max_length

  # x: text 와 y: label 매핑
  def __getitem__(self,index):

    data_point = self.data.iloc[index]
    text = data_point['document'].tolist() # 리스트로 바꿔서 넣기.
    label = data_point['label']
    # text = self.data[index]['document']
    # label = self.data[index]['label']


    if isinstance(text,str) and text.lower() == 'nan':
      # text = "N/A"
      return None, None, None

    # tokenize
    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    # squeeze: 지정된 차원을 제거함.
    input_ids = inputs['input_ids'].squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0)
    # print(input_ids)
    return input_ids, attention_mask, label

  def __len__(self):
    return len(self.data)


In [75]:
train_dataset = CustomDataset(train_data, tokenizer, max_length)
test_dataset   = CustomDataset(test_data, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset,batch_size = batch_size,num_workers=2)
test_dataloader = DataLoader(test_dataset,batch_size = batch_size,num_workers=2)


In [76]:
class BertCLF(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=2,  dr_rate = None,  params=None  ):
    # 상속
    super(BertCLF,self).__init__()
    # bert 모델 불러오기
    self.bert = bert
    # DropOut Rate설정
    self.dr_rate = dr_rate
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)
    # Linear Layer 설정
    self.classifier = nn.Linear(hidden_size,num_classes)

  def forward(self, input_ids, attention_mask):

    outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask) # attention_mask <- custom_dataset(__getitem__)
    # bert가 반환한 outputs에는 hidden state와 pooler라는 튜플을 반환한다.
    # bert 모델에서 반환된 튜플을 올바르게 처리하기 위해 forward 메서드를 수정해야함.
    last_hidden_state = outputs[0]
    pooled_output = outputs[1]

      # cls 토큰 표현의 pooled된 결과물을 추출한다.
    # pooled_output = outputs.pooler_output

    if self.dr_rate:
      pooled_output = self.dropout(pooled_output)

    logits = self.classifier(pooled_output)

    return logits





In [77]:
bert_classifier = BertCLF(bertmodel, dr_rate=dr_rate).to(device)

# Train

In [78]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in bert_classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in bert_classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [79]:
# [define] : loss function, optimizer, epochs, scheduler, ...

loss_function = nn.CrossEntropyLoss()
optimizer = AdamW(optimizer_grouped_parameters,lr = learning_rate)

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케줄러 :  훈련 중에 학습룰 일정을 업데이트 하는데 사용된다.
#             LR을 조정하여 모델 성능이나 학습 안정성을 향상시키는데 사용되는 기술
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [80]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [81]:
for epoch in range(num_epochs):
  bert_classifier.train()
  train_acc = 0.0
  test_acc = 0.0

  for batch_id, batch in enumerate(train_dataloader):

    # batch 객체에서 각 값들을 꺼네 gpu에 보내준다.
    inputs, attention_mask, labels = batch
    inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)

    # forward pass
    out = bert_classifier(inputs, attention_mask)

    # loss 계산
    loss = loss_function(out, labels)

    # backpropagation
      # opimizer를 zero_grad로 초기화후
    optimizer.zero_grad()
      # 기울기 폭주 문제를 방지하고자 사용
    torch.nn.utils.clip_grad_norm_(bert_classifier.parameters(),max_grad_norm)
    loss.backward()
      # loss 최적화
    optimizer.step()
    scheduler.step()

    train_acc += calc_accuracy(out,labels)
    if batch_id % log_interval == 0:
        print("epoch {} batch id {} loss {} train acc {}".format(epoch+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
  print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

  bert_classifier.eval()

  for batch_id, batch in enumerate(test_dataloader):
    inputs, attention_mask, labels = batch
    inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
    out = bert_classifier(inputs,attention_mask)
    test_acc += calc_accuracy(out,labels)

  print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))







epoch 1 batch id 1 loss 0.7256841659545898 train acc 0.453125
epoch 1 batch id 201 loss 0.29704657196998596 train acc 0.7388837064676617
epoch 1 batch id 401 loss 0.36986327171325684 train acc 0.8000701371571073


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-74-44329f06c283>", line 24, in __getitem__
    inputs = self.tokenizer.encode_plus(
  File "/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py", line 2977, in encode_plus
    return self._encode_plus(
  File "/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils.py", line 719, in _encode_plus
    first_ids = get_input_ids(text)
  File "/usr/lo

In [ ]:
입력값 nan은 유효하지 않다, 반드시 문자열, 정수로된 리스트/튜블

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import pandas as pd

data = pd.read_json('/content/drive/MyDrive/data/nlp_dataset/TL_SNS_01.패션/1-1.여성의류(196).json')
data.head()

,Index,RawText,Source,Domain,MainCategory,ProductName,Syllable,Word,GeneralPolarity,Aspects
0,1024338,안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 ...,SNS,패션,여성의류,OO 경량 다운 자켓,513,121,1,"[{'Aspect': '디자인', 'SentimentText': '딱 기본 스타일인..."
1,1024477,드디어 겨울이 찾아왔네요. 이제부터 슬슬 겨울 패딩 장만하셔야지요? 패딩 소개해 드...,SNS,패션,여성의류,OO 아** 구스코트,464,105,1,"[{'Aspect': '사이즈', 'SentimentText': '저는 블랙90 사..."
2,1025044,오늘도 정말 춥네요... 롱패딩 찾고 계신 분을 위한 후기 공유합니다. 키 158...,SNS,패션,여성의류,OO 아** 구스코트,314,78,1,"[{'Aspect': '사이즈', 'SentimentText': '키 158센티로에..."
3,1025046,이웃님들 오늘도 안녕하신가요? 오늘은 따끈한 신상 패딩 후기 올려봅니다~~ 겨울이...,SNS,패션,여성의류,OO 아** 구스코트,307,74,1,"[{'Aspect': '색상', 'SentimentText': '흰색 패딩이 너무나..."
4,1025071,OOO 구스로 소문난 OO의 롱패딩~ 한번 구경 가봐요. 일단 보는 순간 고급스럽...,SNS,패션,여성의류,OO 아** 구스코트,279,68,1,"[{'Aspect': '소재', 'SentimentText': ' 일단 보는 순간 ..."
